# 📈 AV Simulation Visualization Examples

Create interactive visualizations of simulation data.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aanshshah/av-simulation/blob/main/examples/notebooks/04_visualization_examples.ipynb)

## Setup

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

# Configure plotting
plt.style.use('seaborn')
colors = {
    'ego': '#2E8B57',      # Sea Green
    'other': '#FF6347',    # Tomato
    'road': '#808080'      # Gray
}

print('✅ Visualization setup complete')

## Create Plots

In [ ]:
# Sample data visualization
t = np.linspace(0, 10, 100)
speed = 20 + 5 * np.sin(t) + np.random.normal(0, 1, 100)

plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(t, speed, color=colors['ego'], linewidth=2)
plt.title('Vehicle Speed Profile')
plt.xlabel('Time (s)')
plt.ylabel('Speed (m/s)')

print('📊 Plots created successfully')